# SGV 12 Locations

A notebook to create a data sets of images from the collection Ernst Brunner that contain locations. 

In [2]:
import os
import json
from collections import defaultdict

# path to the complete, local, collection data
input_dir = "/Users/maxfrischknecht/mfvk_cloud/A_PhD/07_Data/SGV_12N_Metadata_full"

In [3]:
# loop over all the files in the input_dir
result = []
for file_name in os.listdir(input_dir):
    if file_name.endswith('.json'):
        with open(os.path.join(input_dir, file_name), 'r') as f:
            # the file is open
            data = json.load(f)
            # place to save the extracted data
            reduced_data = []

            for obj in data:
                
                # test if image has id and comment
                if 'schema:identifier' in obj and 'schema:location' in obj:
                    reduced_obj = {}

                    # save the filename as id
                    reduced_obj['schema:identifier'] = obj['schema:identifier'][0]['@value']

                    # loop over the array of locations
                    loc_list = []
                    for item in obj['schema:location']:
                        if 'display_title' in item:
                            loc_list.append(item['display_title'])
                    reduced_obj["schema:location"] = loc_list;
 
                    # save the new objects with its properties in a list
                    # print(reduced_obj)
                    reduced_data.append(reduced_obj)

            # save the list of reduced objects in a global list
            result.extend(reduced_data)

In [4]:
print(result[5])
print(len(result))

{'schema:identifier': 'SGV_12N_07511', 'schema:location': ['Kanton Uri']}
42107


In [5]:
# Aggregating items
aggregated_result = defaultdict(lambda: {'schema:identifier': []})

for item in result:
    key = tuple(item['schema:location'])
    aggregated_result[key]['schema:identifier'].append(item['schema:identifier'])
    aggregated_result[key]['schema:location'] = item['schema:location']

# Convert aggregated_result to a list
final_result = list(aggregated_result.values())
print(len(final_result))
print(final_result[0])

1087
{'schema:identifier': ['SGV_12N_07501', 'SGV_12N_07502', 'SGV_12N_07503', 'SGV_12N_07882', 'SGV_12N_07883', 'SGV_12N_07884', 'SGV_12N_07885', 'SGV_12N_07886', 'SGV_12N_07887', 'SGV_12N_07888', 'SGV_12N_07889', 'SGV_12N_07890', 'SGV_12N_07891', 'SGV_12N_07892', 'SGV_12N_07893', 'SGV_12N_07894', 'SGV_12N_07895', 'SGV_12N_07896', 'SGV_12N_07897', 'SGV_12N_07898', 'SGV_12N_07899', 'SGV_12N_07900', 'SGV_12N_08018', 'SGV_12N_08019', 'SGV_12N_08020', 'SGV_12N_08021', 'SGV_12N_08022', 'SGV_12N_08023', 'SGV_12N_08032', 'SGV_12N_08033', 'SGV_12N_08034', 'SGV_12N_08035', 'SGV_12N_08036', 'SGV_12N_08037', 'SGV_12N_08038', 'SGV_12N_08039', 'SGV_12N_08040', 'SGV_12N_08041', 'SGV_12N_08042', 'SGV_12N_08043', 'SGV_12N_08055', 'SGV_12N_08088', 'SGV_12N_08089', 'SGV_12N_08090', 'SGV_12N_08091', 'SGV_12N_08092', 'SGV_12N_08093', 'SGV_12N_08094', 'SGV_12N_08095', 'SGV_12N_08096', 'SGV_12N_08097', 'SGV_12N_08098', 'SGV_12N_08099', 'SGV_12N_08440', 'SGV_12N_08760', 'SGV_12N_09067', 'SGV_12N_09068', 'SG

In [7]:
with open('./export/sgv-12_locations.json', 'w') as json_file:
    json.dump(final_result, json_file, indent=4)

print("Data has been exported")

Data has been exported
